In [1]:
import numpy as np

import utilities.constants as const
from core.TrajectoryDF import NumPandasTraj as NumTrajDF
from features.spatial_features import SpatialFeatures as spatial
from features.temporal_features import TemporalFeatures as temporal
from utilities.conversions import Conversions as con
from preprocessing.filters import Filters as filters
from utilities.DistanceCalculator import FormulaLog as calc
from features.helper_functions import Helpers as helpers

import pandas as pd
import time
np.seterr(invalid='ignore')
start = time.time()

In [2]:
# %%time
#
# Reading the geolife dataset and converting to NumPandasTraj.
geolife = pd.read_csv('./data/geolife_sample.csv')
geolife = NumTrajDF(geolife,'lat','lon','datetime','id')
#
# Reading the gulls dataset and converting to NumPandasTraj.
gulls = pd.read_csv('./data/gulls.csv')
gulls = NumTrajDF(gulls,
                 latitude='location-lat',
                 longitude='location-long',
                 datetime='timestamp',
                 traj_id='tag-local-identifier',
                 rest_of_columns=[])


# Reading the atlantic dataset, cleaning it up and
# then converting it to NumPandasTraj.
atlantic = pd.read_csv('./data/atlantic.csv')
atlantic = con.convert_directions_to_degree_lat_lon(atlantic, 'Latitude',"Longitude")
def convert_to_datetime(row):
        this_date = '{}-{}-{}'.format(str(row['Date'])[0:4], str(row['Date'])[4:6], str(row['Date'])[6:])
        this_time = '{:02d}:{:02d}:00'.format(int(row['Time']/100), int(str(row['Time'])[-2:]))
        return '{} {}'.format(this_date, this_time)
atlantic['DateTime'] = atlantic.apply(convert_to_datetime, axis=1)
atlantic = NumTrajDF(atlantic,
                         latitude='Latitude',
                         longitude='Longitude',
                         datetime='DateTime',
                         traj_id='ID',
                         rest_of_columns=[])
atlantic = atlantic.drop(columns='Time')
atlantic = temporal.create_date_column(atlantic)
atlantic.head()

Name        Date Event Status  \
traj_id  DateTime                                                            
AL011851 1851-06-25 00:00:00              UNNAMED  1851-06-25           HU   
         1851-06-25 06:00:00              UNNAMED  1851-06-25           HU   
         1851-06-25 12:00:00              UNNAMED  1851-06-25           HU   
         1851-06-25 18:00:00              UNNAMED  1851-06-25           HU   
         1851-06-25 21:00:00              UNNAMED  1851-06-25     L     HU   

                               lat   lon  Maximum Wind  Minimum Pressure  \
traj_id  DateTime                                                          
AL011851 1851-06-25 00:00:00  28.0 -94.8            80              -999   
         1851-06-25 06:00:00  28.0 -95.4            80              -999   
         1851-06-25 12:00:00  28.0 -96.0            80              -999   
         1851-06-25 18:00:00  28.1 -96.5            80              -999   
         1851-06-25 21:00:00  28.2 -96.8            80              -999   

                              Low Wind NE  Low Wind SE  Low Wind SW  \
traj_id  DateTime                                                     
AL011851 1851-06-25 00:00:00         -999         -999         -999   
         1851-06-25 06:00:00         -999         -999         -999   
         1851-06-25 12:00:00         -999         -999         -999   
         1851-06-25 18:00:00         -999         -999         -999   
         1851-06-25 21:00:00         -999         -999         -999   

                              Low Wind NW  Moderate Wind NE  Moderate Wind SE  \
traj_id  DateTime                                                               
AL011851 1851-06-25 00:00:00         -999              -999              -999   
         1851-06-25 06:00:00         -999              -999              -999   
         1851-06-25 12:00:00         -999              -999              -999   
         1851-06-25 18:00:00         -999              -999              -999   
         1851-06-25 21:00:00         -999              -999              -999   

                              Moderate Wind SW  Moderate Wind NW  \
traj_id  DateTime                                                  
AL011851 1851-06-25 00:00:00              -999              -999   
         1851-06-25 06:00:00              -999              -999   
         1851-06-25 12:00:00              -999              -999   
         1851-06-25 18:00:00              -999              -999   
         1851-06-25 21:00:00              -999              -999   

                              High Wind NE  High Wind SE  High Wind SW  \
traj_id  DateTime                                                        
AL011851 1851-06-25 00:00:00          -999          -999          -999   
         1851-06-25 06:00:00          -999          -999          -999   
         1851-06-25 12:00:00          -999          -999          -999   
         1851-06-25 18:00:00          -999          -999          -999   
         1851-06-25 21:00:00          -999          -999          -999   

                              High Wind NW  
traj_id  DateTime                           
AL011851 1851-06-25 00:00:00          -999  
         1851-06-25 06:00:00          -999  
         1851-06-25 12:00:00          -999  
         1851-06-25 18:00:00          -999  
         1851-06-25 21:00:00          -999

In [3]:
# %%time

#Now, let create a bounding box of 100 km radius from the
#coordinates 39, 116.

bbox = filters.get_bounding_box_by_radius(39, 116, 100000)
bbox

(38.100678394081264, 114.84275815636957, 39.89932160591873, 117.15724184363044)

In [4]:
# Now, lets filter the trajectory based on date. We will
# try all the possible combinations for the filtering.

small = filters.filter_by_date(atlantic, start_date='1851-06-25',end_date='2011-01-01')
print(f"Length of atlantic: {len(atlantic)}")
print(f"Length of small: {len(small)}")

Length of atlantic: 49105
Length of small: 46909


In [5]:
# Now, lets filter the trajectory based on datetime. We will
# try all the possible cases one by one.

tiny = filters.filter_by_datetime(atlantic, start_dateTime='1859-09-21 23:00:00' ,
                                  end_dateTime='2011-09-21 23:00:00')
print(f"Length of atlantic: {len(atlantic)}")
print(f"Length of tiny: {len(tiny)}")

Length of atlantic: 49105
Length of tiny: 46536


In [6]:
# Now, lets filter the dataframe based on maximum speed.

atlantic = spatial.create_speed_from_prev_column(atlantic)
max_speed_filt_df = filters.filter_by_max_speed(atlantic, 10)
print(f"Length of atlantic: {len(atlantic)}")
print(f"Length of speed_filt_df: {len(max_speed_filt_df)}")

Length of atlantic: 49105
Length of speed_filt_df: 41358


In [7]:
# Now, lets filter the dataframe based on minimum speed.

min_speed_filt = filters.filter_by_min_speed(max_speed_filt_df, 5)
print(f"Length of speed_filt_df: {len(max_speed_filt_df)}")
print(f"Length of min_speed_filt: {len(min_speed_filt)}")

Length of speed_filt_df: 41358
Length of min_speed_filt: 20485


In [8]:
# Now, lets filter the dataframe based on minimum distance
# between consecutive points.

min_distance_filt = filters.filter_by_min_consecutive_distance(atlantic,
                                                               125000)
print(f"length of atlantic: {len(atlantic)}")
print(f"length of min_distance_filt: {len(min_distance_filt)}")

length of atlantic: 49105
length of min_distance_filt: 20584


In [9]:
# Now, lets filter the dataframe based on maximum distance
# between consecutive points.

max_distance_filt = filters.filter_by_max_consecutive_distance(min_distance_filt,
                                                               500000)
print(f"length of min_distance_filt: {len(min_distance_filt)}")
print(f"length of max_distance_filt: {len(max_distance_filt)}")

length of min_distance_filt: 20584
length of max_distance_filt: 20412


In [10]:
# Now, lets filter the data based on maximum speed as
# well as maximum distance between 2 consecutive points.

max_dist_speed_filt = \
    filters.filter_by_max_distance_and_speed(atlantic, max_distance=300000, max_speed=5)
print(f"length of atlantic: {len(atlantic)}")
print(f"length of max_dist_speed_filt: {len(max_dist_speed_filt)}")

length of atlantic: 49105
length of max_dist_speed_filt: 20873


In [11]:

# Now, lets filter the data based on minimum speed as
# well as minimum distance between 2 consecutive points.

min_dist_speed_filt = \
    filters.filter_by_min_distance_and_speed(atlantic, min_distance=150000, min_speed=10)
print(f"length of atlantic: {len(atlantic)}")
print(f"length of max_dist_speed_filt: {len(min_dist_speed_filt)}")

length of atlantic: 49105
length of max_dist_speed_filt: 5773


In [12]:
# Now, lets remove the outliers based on the
# distance between 2 consecutive points.

geolife = spatial.create_speed_from_prev_column(geolife)
outlier_df = filters.filter_outliers_by_consecutive_distance(geolife)
print(f"length of geolife: {len(geolife)}")
print(f"length of outlier_df: {len(outlier_df)}")
print(f"Number of outliers: {len(geolife) - len(outlier_df)}")

length of geolife: 217653
length of outlier_df: 212126
Number of outliers: 5527


In [13]:
odf_two = filters.filter_outliers_by_consecutive_speed(geolife)
print(f"length of geolife: {len(geolife)}")
print(f"length of outlier_df: {len(odf_two)}")
print(f"Number of outliers: {len(geolife) - len(odf_two)}")

length of geolife: 217653
length of outlier_df: 195283
Number of outliers: 22370


In [14]:
# Now, lets remove the trajectories that have
# fewer than 5 points.

short_traj_gone = filters.remove_trajectories_with_less_points(atlantic, 5)
print(f"Number of unique Traj IDs in atlantic: {atlantic.traj_id.nunique()}")
print(f"Number of unique Traj IDs left after filter: {short_traj_gone.traj_id.nunique()}")



Number of unique Traj IDs in atlantic: 1814
Number of unique Traj IDs left after filter: 1772
